In [1]:
import requests as req
from bs4 import BeautifulSoup as bsp
import csv
import time
from selenium import webdriver
import random
import requests

In [2]:
info = {  # uninitialized dictionary to use for later
    'Name': [],
    'Price': [],
    'Image': [],
    'Link': []
}

In [3]:
url = 'https://lamaretail.com/collections/woman-studio-collection'

In [4]:
# Getting the navbar of the website
soup = bsp(req.get(url).content, 'html.parser')
# getting the ul tag with the class name "site-nav site-navigation small--hide"
ol = soup.find('ul', class_='site-nav site-navigation small--hide')
# getting the li tags inside the ul tag
lis = ol.find_all('li')
# within each li tag there is an ul tag with the class name "site-nav__dropdown text-left" and within that there are li tags
# we are extracting the href attribute of the a tag within the li tag
links = []
for li in lis:
    ul = li.find('ul', class_='site-nav__dropdown text-left')
    if ul is not None:
        for li in ul.find_all('li'):
            a = li.find('a')
            links.append(a['href'])


In [5]:
for i in range(len(links)):
    links[i] = 'https://lamaretail.com' + links[i]

print(links)

['https://lamaretail.com/collections/woman-apparel-new-in', 'https://lamaretail.com/collections/woman-ss-24-all', 'https://lamaretail.com/collections/woman-studio-collection', 'https://lamaretail.com/collections/true-body', 'https://lamaretail.com/collections/tops-blouses', 'https://lamaretail.com/collections/woman-t-shirts', 'https://lamaretail.com/collections/woman-dresses', 'https://lamaretail.com/collections/woman-pants', 'https://lamaretail.com/collections/woman-jeans', 'https://lamaretail.com/collections/woman-camisole-bandeaus', 'https://lamaretail.com/collections/woman-bodysuits', 'https://lamaretail.com/collections/woman-blazers', 'https://lamaretail.com/collections/woman-hoodies-sweatshirt', 'https://lamaretail.com/collections/woman-sweaters-cardigans', 'https://lamaretail.com/collections/woman-jackets-coats', 'https://lamaretail.com/collections/man-apparel-new-in', 'https://lamaretail.com/collections/man-ss-24-all', 'https://lamaretail.com/collections/man-shirts', 'https://l

In [6]:
def get_valid_proxies():
    proxy_list_url = 'https://free-proxy-list.net/'
    response = requests.get(proxy_list_url)
    soup = bsp(response.text, 'html.parser')
    proxy_data = []
    rows = soup.find_all('tr')[1:]
    for row in rows:
        columns = row.find_all('td')
        if len(columns) >= 8:
            ip_address = columns[0].text.strip()
            google_enabled = columns[5].text.strip().lower() == 'yes'
            https_enabled = columns[6].text.strip().lower() == 'yes'
            last_checked = columns[7].text.strip()
            if (last_checked.endswith('mins ago') and int(last_checked.split(' ')[0]) < 15) or last_checked.endswith('hours ago'):
                if google_enabled or https_enabled:
                    proxy_data.append({'ip_address': ip_address, 'google_enabled': google_enabled, 'https_enabled': https_enabled})

    return proxy_data

def rotate_user_agent(proxy):
    if proxy:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
            'http': f'http://{proxy}',
            'https': f'https://{proxy}'
        }
    else:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
        }
    return headers


In [7]:
# Get a list of valid proxies
proxies = get_valid_proxies()

# Function to get a random proxy from the list
def get_random_proxy():
    return random.choice(proxies)['ip_address']


In [8]:
# Scrapping the links one by one
for link in links:
    proxy = get_random_proxy()
    headers = rotate_user_agent(proxy)

    # Getting the navbar of the website
    soup = bsp(req.get(link, headers=headers, proxies={"http": proxy, "https": proxy}).content, 'html.parser')
    
    # scrolling the page to the bottom above the footer then back up to load all the items until the end
    driver = webdriver.Chrome()
    driver.get(link)
    
    Previous_Height = driver.execute_script("return document.body.scrollHeight")
    while True:
        # It will scroll to right above the footer of the page then scoll to the top then back to the bottom untill there is no new items being loaded
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight - 1000);")
        time.sleep(1)
        driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(1)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight - 1000);")
        time.sleep(1)
        New_Height = driver.execute_script("return document.body.scrollHeight")
        if New_Height == Previous_Height:
            break
        Previous_Height = New_Height
        # if there is a popup press the close button
        try:
            driver.find_element_by_class_name('modal-close').click()
        except:
            pass
        
    # getting the html content of the page
    html = driver.page_source
    driver.quit()
    # parsing the html content
    soup = bsp(html, 'html.parser')
    # getting the image form div class "grid-product__image-mask"
    images = soup.find_all('div', class_='grid-product__image-mask')
    # getting the name from div class "grid-product__title grid-product__title--body"
    names = soup.find_all('div', class_='grid-product__title grid-product__title--body')
    # getting the price from span class "grid-product__price"
    prices = soup.find_all('span', class_='grid-product__price')
    # getting the link from a tag class "grid-product__link"
    links = soup.find_all('a', class_='grid-product__link')
    
    # adding to the dictionary
    info['Name'] += [name.text.strip() for name in names]
    info['Price'] += [price.text.strip() for price in prices]
    info['Image'] += [image.find('img')['src'] for image in images]
    info['Link'] += ['https://lamaretail.com' + link['href'] for link in links]


ProxyError: HTTPSConnectionPool(host='lamaretail.com', port=443): Max retries exceeded with url: /collections/woman-apparel-new-in (Caused by ProxyError('Unable to connect to proxy', OSError('Tunnel connection failed: 400 Bad Request')))

In [ ]:
# Write dictionary to CSV file
csvFile = 'Lama.csv'  # writing the headings of the columns
with open(csvFile, mode='w', newline='', encoding='utf-8') as file:  # using an encoder for special characters
    fieldnames = list(info.keys())  # typecasting the keys into a list
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()

    for i in range(len(info['Name'])):  # according to number of items, we are copying every value of every key into its respective heading
        row = {}
        for key in info.keys():
            row[key] = info[key][i]

        writer.writerow(row)
